<a href="https://colab.research.google.com/github/szkjiro/program/blob/main/PyWord2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ワードクラウドの作成において，以前に形態素解析を扱いました．
次のプログラムはMeCabを用いて例文「失敗に挑戦しないことが最大の失敗である」
の形態素解析の実行部分だけを抜き出しています．
MeCabを使うために，最初の行はMeCab辞書をインストールしています．

In [ ]:
!pip install mecab-python3 unidic-lite
import MeCab
# parse は，文章の文法解析をする一般的なことば
print(MeCab.Tagger().parse("失敗に挑戦しないことが最大の失敗である"))

自然言語処理のためによく使われる表現に**One-hotベクトル**があります．
上の解析例でいえば，次のような0と1からなる数の並び（ベクトル）を対応させます．
1が一回だけ現れ，残りは0であることからつけられた名前です．
見てわかるように「何番目に何という単語がある」という情報をベクトルの形に書き換えているだけです．
形態素解析の切り出した単語は全部で12個です．

- 失敗 (1,0,0,0,0,0,0,0,0,0,0,0)
- に (0,1,0,0,0,0,0,0,0,0,0,0)
- 挑戦 (0,0,1,0,0,0,0,0,0,0,0,0)
- し (0,0,0,1,0,0,0,0,0,0,0,0)
- ない (0,0,0,0,1,0,0,0,0,0,0,0)
- こと (0,0,0,0,0,1,0,0,0,0,0,0)
- が (0,0,0,0,0,0,1,0,0,0,0,0)
- 最大 (0,0,0,0,0,0,0,1,0,0,0,0)
- の (0,0,0,0,0,0,0,0,1,0,0,0)
- 失敗 (0,0,0,0,0,0,0,0,0,1,0,0)
- で (0,0,0,0,0,0,0,0,0,0,1,0)
- ある (0,0,0,0,0,0,0,0,0,0,0,1)

同じ単語「失敗」でも，現れる位置により異なるベクトルに作られています．
このような単語と位置の組み合わせから，**n-gram**を作成することもできます．
n-gramは隣接するn個の単語が何回登場するかの統計情報を表すものです．

## 共起関係とコーパス

このOne-hotベクトル表現は，扱う文章が長くなればなるほど，ベクトル（の数の並び）も長くなります．
n-gramだけでは単語もしくは複数の単語の並びの統計情報だけなので，
たとえば「失敗という単語から10語以内に成功という単語が現れやすい
（仮の主張なので本当ではないと思います）」のようなことはわかりません．
ここに書いた何語以内に特定の単語同士が現れやすい関係を**共起関係**といい，
言語学者はそうした情報を整備した**コーパス（辞書）**を従来から整備してきました．

## 分散表現

One-hotベクトルのような言語ベクトルから，単語だけをうまく扱う方法はないのでしょうか．
そこで考えられたものの一つが分散表現です．
分散表現では意味の似た単語はベクトルとして近くに位置し，
似てないものは遠くに位置するものと考えます．
またベクトル長は固定長であり，計算処理も扱いやすくします．

このような都合のよいベクトルを実現する方法があるのでしょうか．
その方法の一つが，ニューラルネットワークによる機械学習を通じて，
One-hotベクトルから分散表現を作り出す**Word2Vec**でした．
Word2vecは，Googleの研究者であるトマス・ミコロフの研究チームにより，
2013年に公開されました．
そこではスキップグラム法（skip-gram法）およびCBOW(continuous bag-of-words)という2つの技術を組み合わせています．
いずれも教師あり機械学習が用いられています．

これを実行する準備に，日本語Wikipediaから作成済みの言語モデルなどをインストールします．2分以上かかるでしょう．

In [ ]:
!pip install transformers[ja]
!wget https://github.com/singletongue/WikiEntVec/releases/download/20190520/jawiki.word_vectors.200d.txt.bz2
!bzip2 -d jawiki.word_vectors.200d.txt.bz2

言語モデルはgensimというライブラリを利用します．
2行目の右辺になんとか.txtというファイル名があります．
これがWikipediaから作成されたテキスト情報です．
きわめて巨大なデータファイルなので，表示させるとたいへんなことになります．2分以上かかるでしょう．

In [3]:
from gensim import models
w2v_model =  models.KeyedVectors.load_word2vec_format('jawiki.word_vectors.200d.txt', binary=False)

信州という単語の内容を見てみましょう．

In [ ]:
word = "信州"

# Word2Vecで作成した単語ベクトルの値
word_vec = w2v_model.__getitem__(word)
print(word_vec.shape)

# Word2Vecで作成した単語ベクトル
print(word_vec)

次では信州に似ている単語を順に表示させます．

In [ ]:
Similars = w2v_model.most_similar(positive=['信州'])
for Niteiru in Similars:
    print(Niteiru)

各単語の右に現れた数字は基準とした単語「信州」との相関係数と呼んでいいものです．
すでに観察した信州に対する小数値の並びのベクトル同士の内積の値でもあります．

2つのベクトル$a$と$b$のなす角が$\theta$であるとき，その内積に関して
$$
a\cdot b = |a|\cdot |b|\cos\theta
$$
という関係式があることを覚えている人はいるでしょう．
この式を変形すれば
$$
\cos\theta = \frac{a\cdot b}{|a|\cdot |b|}
$$
となり，この式が2つのベクトルの相関係数を与えています．
この式にもとづき，上で見た相関係数を**コサイン類似度**とも呼びます．
ベクトルのことばでいうと，2つのベクトルが似ているほど，2つのベクトルのなす角度が0に近いのです．

この類似単語の表示は，複数の単語に対しても可能です．

In [ ]:
Similars = w2v_model.most_similar(positive=['長野県', 'サッカー'])
for Niteiru in Similars:
    print(Niteiru)

単語同士の演算もできます．
ベクトルで表されているからです．
positiveに指定した単語から，negativeに指定した単語（複数指定可能，上の例を参考）をベクトルとして計算した結果を類似度順に表示します．

In [ ]:
Keisan = w2v_model.most_similar(positive=['信州','グルメ'], negative=['ラーメン'])
for Atai in Keisan:
    print(Atai)

## ChatGPTとは何なのか

現在，世界的にChatGPTの利用に注目が集まっています．
ChatGPTは大規模言語モデル（LLM; Large-scale Language Model）の一つGPT (Generative Pre-trained Transformer) をもとに開発されました．
TransformerはWord2Vecの欠点を改良する注意機構という仕組みであり，2017年にGoogleが公開しました．
これはGoogleが開発したLLMであるBERTに組み込まれています．
このためLLMに関する公開情報ではBERTに関する解説（書籍もあり）が，最も包括的に書かれています．
上のインストールの仕方にtransformerの名前があったのは，現在では，Word2Vecがその一部として提供されているためです．

大きな改良がなされたとはいえ，GPTはWord2Vecをもとにした分散表現の一種です．
このためChatGPTが綴る文章は，上の例題で扱ってきた確率にしたがって文章を並べる仕組みなのです．